# Python training SQL

Some basic questions:
- In this database, show the customers, products, finally the orders. 
- Display customers that put stuff in their carts, who ordered, pending carts.

Calculations:
- In this database, display the total number of customers, products and orders.
- In this database, show the number of pending carts. 
- In this database, display the total orders amount (let's change the times, 1 month)
- In this database, find the top 5 customers (ordered by spending amount)
- In this database find the flop 5 customers (ordered by spending amount)
- In this database, find the top 5 products. (ordered most, in quantity)
- In this database, find the average order, product's price.
- In this database, find the median order, product price.
- In this database, find ordered amount quartils
- In this database, find max et min total amount order
- In this database, find max et min total quantity ordered
- In this database, find max and min price products.
- Find all pending carts accumulated values

Introduce Windows Functions
- Find the best amount spent by customer (max 1 order)
- Find all pending carts accumulated values
- For customers who ordered, display spending total amounts ranking by deciles
- Find per customer, the transactions ranks
- Let's rank ordered products by quantity send
- Get the average order per group of customer total spending


Mix all queries
- Temporary column: Product's price classes (low < 33 < medium < 66 < high)
- Find the amount spend per customer
- Find the most ordered categories
- Display the total profits per category 

### In this database, show the customers, products, finally the orders. 

In [ ]:
USE `Test_SQL`;

SELECT * FROM customers;
SELECT * FROM products;
SELECT * FROM orders; 

### In this database, display customers who ordered,  also with pending carts.

In [ ]:
USE `Test_SQL`;

SELECT DISTINCT C.customer_id FROM carts as C; -- with carts
SELECT C.customer_id FROM customers as C; -- the total

SELECT DISTINCT C.customer_id, count(O.cart_id) -- every customers, counting the number of times, that ordered
FROM orders as O 
OUTER JOIN carts as C 
ON C.cart_id = O.cart_id
GROUP BY C.customer_id
ORDER BY C.customer_id;

SELECT DISTINCT C.customer_id, count(C.cart_id) -- all carts
FROM carts as C 
GROUP BY C.customer_id
ORDER BY C.customer_id;

SELECT DISTINCT C.customer_id, count(C.cart_id) -- the pending carts
FROM carts as C 
WHERE C.total = 0
GROUP BY C.customer_id
ORDER BY C.customer_id;

# Calculations

### In this database, display the total number of customers, products and orders.

In [ ]:
USE `Test_SQL`;

SELECT count(C.customer_id) FROM customers as C;
SELECT count(O.order_id) FROM orders as O;
SELECT count(P.product_id) FROM products as P;

### In this database, show the number of pending carts. 

In [ ]:
-- all pending carts
WITH  pending_carts AS 
	(SELECT DISTINCT C.customer_id, count(C.cart_id) as cart_count
	FROM carts as C 
	WHERE C.total = 0
	GROUP BY C.customer_id
	ORDER BY C.customer_id) 
		SELECT sum(cart_count)
        FROM pending_carts;


### In this database, display the total orders amount (let's change the times, 1 month)

In [ ]:
SELECT sum(C.total) 
FROM carts as C, orders as O 
WHERE C.total > 0 AND O.cart_id = C.cart_id;

### In this database, find the top 5 customers (ordered by spending amount)

In [ ]:
SELECT C.customer_id, SUM(C.total)
FROM carts as C, orders as O
WHERE O.cart_id = C.cart_id
GROUP BY C.customer_id
ORDER BY -SUM(C.total)
LIMIT 5;

### In this database find the flop 5 customers (ordered by spending amount)

In [ ]:
SELECT C.customer_id, SUM(C.total)
FROM carts as C, orders as O
WHERE O.cart_id = C.cart_id
GROUP BY C.customer_id
ORDER BY SUM(C.total) -- delete the "-"
LIMIT 5;

### In this database, find the top 5 products. (ordered most, in quantity)

In [ ]:
# What is the fastest method?

SELECT P.product_id, P.product_name, SUM(Joint.quantity)
FROM 
    carts as C, 
    orders as O, 
    cart_products as Joint, 
    products as P
WHERE 
    O.cart_id = C.cart_id 
    AND C.total > 0 
    AND C.cart_id = Joint.cart_id 
    AND Joint.product_id = P.product_id
GROUP BY 
    P.product_name, 
    P.product_id
ORDER BY 
    -SUM(Joint.quantity)
LIMIT 5;

# Timing (execution time): 0.00198603 s

SELECT P.product_id, P.product_name, SUM(Joint.quantity)
	FROM products as P
	INNER JOIN cart_products as Joint
    ON Joint.product_id = P.product_id
		INNER JOIN carts as C
        ON C.cart_id = Joint.cart_id
			INNER JOIN orders as O
            ON O.cart_id = C.cart_id
				WHERE C.total > 0
                GROUP BY P.product_name, P.product_id
				ORDER BY -SUM(Joint.quantity)
				LIMIT 5;
                
# Timing (execution time): 0.00216079 s (time +8,79% )


### In this database, find the average order, product's price.

In [ ]:
SELECT C.customer_id, AVG(C.total)
FROM carts as C, orders as O
WHERE O.cart_id = C.cart_id AND C.total > 0
GROUP BY C.customer_id
ORDER BY -AVG(C.total);

SELECT AVG(P.price)
FROM products as P

### In this database, find the median order, product price.

In [ ]:
#https://www.sisense.com/blog/medians-in-sql/
set @ct := (select count(1) from carts);
set @row_id := 0;

select avg(truc.total) as median
from (select * from carts order by carts.total) as truc, orders
where (select @row_id := @row_id + 1 as machin) AND truc.total > 0 AND truc.cart_id = orders.cart_id
between @ct/2.0 and @ct/2.0 + 1;

# https://stackoverflow.com/questions/1291152/simple-way-to-calculate-median-with-mysql
SELECT CAST(SUBSTRING_INDEX(SUBSTRING_INDEX(
 GROUP_CONCAT(total ORDER BY total SEPARATOR ','),
  ',', 50/100 * COUNT(*) + 1), ',', -1) AS DECIMAL) AS `Median`
FROM carts, orders
WHERE carts.total <> 0 AND carts.cart_id = orders.cart_id;


#median price
set @ct := (select count(1) from products);
set @row_id := 0;

select avg(truc.price) as median
from (select * from products order by products.price) as truc
where (select @row_id := @row_id + 1 as machin) AND truc.price > 0
between @ct/2.0 and @ct/2.0 + 1;

### In this database, find ordered amount quartils

In [ ]:
# in orders
SELECT CAST(SUBSTRING_INDEX(SUBSTRING_INDEX(
 GROUP_CONCAT(total ORDER BY total SEPARATOR ','),
  ',', 25/100 * COUNT(*) + 1), ',', -1) AS DECIMAL) AS `Median`
FROM carts, orders
WHERE carts.total <> 0 AND carts.cart_id = orders.cart_id;

SELECT CAST(SUBSTRING_INDEX(SUBSTRING_INDEX(
 GROUP_CONCAT(total ORDER BY total SEPARATOR ','),
  ',', 75/100 * COUNT(*) + 1), ',', -1) AS DECIMAL) AS `Median`
FROM carts, orders
WHERE carts.total <> 0 AND carts.cart_id = orders.cart_id;

### In this database, find max et min total amount order

In [ ]:
SELECT MAX(total_amounts.amount_sum)
FROM (
	SELECT C.customer_id, SUM(C.total) as amount_sum
	FROM carts as C, orders as O
	WHERE O.cart_id = C.cart_id
	GROUP BY C.customer_id
	ORDER BY SUM(C.total) ) as total_amounts

SELECT MIN(total_amounts.amount_sum)
FROM (
	SELECT C.customer_id, SUM(C.total) as amount_sum
	FROM carts as C, orders as O
	WHERE O.cart_id = C.cart_id
	GROUP BY C.customer_id
	ORDER BY SUM(C.total) ) as total_amounts

### In this database, find max et min total quantity ordered

In [ ]:
SELECT MAX(Ordered_Quantity.sum_quant)
FROM (
	SELECT P.product_id, P.product_name, SUM(Joint.quantity) as sum_quant
	FROM 
		carts as C, 
		orders as O, 
		cart_products as Joint, 
		products as P
	WHERE 
		O.cart_id = C.cart_id 
		AND C.total > 0 
		AND C.cart_id = Joint.cart_id 
		AND Joint.product_id = P.product_id
	GROUP BY 
		P.product_name, 
		P.product_id
	ORDER BY 
		-SUM(Joint.quantity) ) as Ordered_Quantity

SELECT MIN(Ordered_Quantity.sum_quant)
FROM (
	SELECT P.product_id, P.product_name, SUM(Joint.quantity) as sum_quant
	FROM 
		carts as C, 
		orders as O, 
		cart_products as Joint, 
		products as P
	WHERE 
		O.cart_id = C.cart_id 
		AND C.total > 0 
		AND C.cart_id = Joint.cart_id 
		AND Joint.product_id = P.product_id
	GROUP BY 
		P.product_name, 
		P.product_id
	ORDER BY 
		-SUM(Joint.quantity) ) as Ordered_Quantity

### In this database, find max and min price products.

In [ ]:
SELECT MAX(P.price)
FROM products as P;

#This doesn't show the name an id... 

# writing MIN() in this query is actually useless, P.price alone would work
SELECT P.product_id as p_id, P.product_name as p_name, MIN(P.price) as p_price
FROM products as P
GROUP BY p_id, p_name
ORDER BY p_price
LIMIT 1;

#In this one we have id and name, the thing is: and the speed?
# MIN is same speed execution MAX
# only MAX: 0.000499 s 
# lot of informations: 0.000612 s (+22%)
# a bit expected, when we want more infos the speed decreases

# Introduce Windows Functions

In [ ]:
#orders total amount along the carts involved in orders

SELECT C.cart_id, SUM(C.total) OVER () cart_amounts
FROM orders as O, carts as C
WHERE C.cart_id = O.cart_id

# customer former orders ranking with only ranks number 5 to 15
SELECT * FROM (
	SELECT 
		ROW_NUMBER() OVER (
			ORDER BY C.total DESC
			) ranking, 
		Cstm.customer_name, 
		Cstm.customer_id, 
		C.cart_id, 
		C.total
	FROM 
		orders as O, 
		carts as C, 
		customers as Cstm
	WHERE C.cart_id = O.cart_id 
		and Cstm.customer_id = C.customer_id ) as test_row
WHERE ranking >= 5 AND ranking <= 15;



### Find the best amount spent by customer (max 1 order)

In [ ]:
WITH sub as (
	SELECT 
            Cstm.customer_name,
            ROW_NUMBER() OVER (
                PARTITION BY -C.total
                ORDER BY C.total) row_num, 
            C.total
        FROM 
            customers as Cstm
                INNER JOIN carts as C
                ON Cstm.customer_id = C.customer_id
                    INNER JOIN orders as O
                    ON C.cart_id = O.cart_id
		WHERE 
			C.total > 0)
SELECT 
    customer_name,
    total,
    row_num
FROM 
	sub
WHERE 
    row_num = 1;

### Find all pending carts accumulated values

In [ ]:
# carts pendings (https://www.tutorialspoint.com/what-is-the-equivalent-of-except-in-mysql)
SELECT 
	C.cart_id
FROM 
	carts as C
WHERE 
	C.cart_id NOT IN (
		SELECT O.cart_id
		FROM orders as O);

# with EXCEPT (that dosn't work in  MySQL)
    SELECT C.cart_id
    FROM carts as C
EXCEPT
	SELECT O.cart_id
	FROM orders as O
    
# order the carts (not ordered yet) by pending amount, with the total who's pending
SELECT 
	C.cart_id,
    C.total,
    SUM(C.total) OVER () AS amount_pending
FROM 
	carts as C
WHERE 
	C.cart_id NOT IN (
		SELECT O.cart_id
		FROM orders as O)
ORDER BY total DESC

### For customers who ordered, display spending total amounts ranking by deciles

In [ ]:
#https://stackoverflow.com/questions/40594466/can-i-group-by-in-sql-query-with-window-function
WITH spendings AS (
	SELECT 
		C.customer_name, 
		MAX(SUM(Ct.total)) OVER (PARTITION BY C.customer_id ) AS order_per_cust
    FROM 
		customers as C,
		carts as Ct,
        orders AS O
	WHERE
		C.customer_id = Ct.customer_id
        AND Ct.cart_id = O.cart_id
	GROUP BY C.customer_id
    ORDER BY order_per_cust DESC
)
SELECT 
    spendings.customer_name, 
    order_per_cust, 
    NTILE (10) OVER ( ORDER BY  order_per_cust DESC) AS deciles
FROM spendings

### Find per customer, the transactions ranks

In [ ]:
SELECT C.customer_name, Ct.total, RANK () OVER (ORDER BY Ct.total DESC ) as amount_rank
FROM
	customers as C,
    carts AS Ct,
    orders AS O
WHERE 
    C.customer_id = Ct.customer_id 
    AND Ct.cart_id = O.cart_id

### Let's rank ordered products by quantity send

In [ ]:
SELECT P.product_id, P.product_name, SUM(Joint.quantity) as sum_quant_send, RANK () OVER ( ORDER BY SUM(Joint.quantity) DESC) AS rank_quantities
FROM 
	carts as C, 
	orders as O, 
	cart_products as Joint, 
	products as P
WHERE 
	O.cart_id = C.cart_id 
	AND C.total > 0 
	AND C.cart_id = Joint.cart_id 
	AND Joint.product_id = P.product_id
GROUP BY 
	P.product_name, 
	P.product_id
ORDER BY 
	-SUM(Joint.quantity)

### Get the average order per group of customer total spending

In [ ]:
SELECT 
	C.customer_name, 
    Ct.total,
	AVG(Ct.total) OVER (PARTITION BY C.customer_id ORDER BY C.customer_id ) AS avg_spending_per_cust
FROM 
	customers as C,
	carts as Ct,
    orders AS O
WHERE
	C.customer_id = Ct.customer_id
    AND Ct.cart_id = O.cart_id
ORDER BY avg_spending_per_cust DESC


# Mix all queries

### Temporary column: Product's price classes (low < 33 < medium < 66 < high)

In [ ]:
#https://datatofish.com/case-statement-sql-server/
SELECT 
	P.product_id,
    P.product_name, 
    P.price, 
    CASE WHEN P.price < 33 THEN "Low"
		WHEN P.price > 66 THEN "High"
        ELSE "Medium"
        END AS products_cat
FROM products AS P
ORDER BY P.price

### Find the amount spend per customer

In [ ]:
WITH u_cat as (
	SELECT 
		P.product_id,
		P.product_name, 
		P.price, 
		CASE WHEN P.price < 33 THEN "Low"
			WHEN P.price > 66 THEN "High"
			ELSE "Medium"
			END AS products_cat
	FROM products AS P
    ),
	join_table AS (
	SELECT
		
		C.customer_name, 
        P.product_id, 
        P.product_name, 
        P.price, 
        Cp.quantity, 
        P.products_cat, 
        Ct.cart_id,
        O.order_id,
        Ct.total
	FROM 
		u_cat as P,
		cart_products as Cp,
		carts as Ct,
		customers as C,
		orders as O
	WHERE
		Ct.cart_id = O.order_id
		and C.customer_id = Ct.customer_id
		and P.product_id = Cp.product_id
		and Cp.cart_id = Ct.cart_id
		)
SELECT 
	join_table.customer_name,
    join_table.order_id, 
    join_table.total, SUM(join_table.total) OVER (PARTITION BY join_table.customer_name) as amount_per_cust
FROM join_table
GROUP BY join_table.order_id

### Find the most ordered categories

In [ ]:
# qt_prd_ord quantity per product ordered, price_prod_cat to make product categories
WITH qt_prd_ord AS (
	SELECT P.product_id, MAX(SUM(Cp.quantity)) OVER (PARTITION BY P.product_id ORDER BY P.product_id) as ordered_prod
	FROM
		carts as Ct,
		orders as O,
		cart_products as Cp,
		products as P
	WHERE
		Ct.cart_id = O.cart_id
		and P.product_id = Cp.product_id
		and Cp.cart_id = Ct.cart_id
	GROUP BY
		P.product_id
	ORDER BY
		ordered_prod
	),

    price_prod_cat as (
    SELECT 
	P.product_id,
    P.product_name, 
    P.price, 
    CASE WHEN P.price < 33 THEN "Low"
		WHEN P.price > 66 THEN "High"
        ELSE "Medium"
        END AS products_cat
	FROM products AS P
	ORDER BY P.price
    )
    
SELECT price_prod_cat.product_id, price_prod_cat.product_name, price_prod_cat.price, price_prod_cat.products_cat,  p_id_qt.ordered_prod
FROM 
	qt_prd_ord,
    price_prod_cat
WHERE
	qt_prd_ord.product_id = price_prod_cat.product_id
ORDER BY 
	qt_prd_ord.ordered_prod DESC
    
    
-- join  version ?

-- sub query version ?

# ordered amount and total quantities per customer ?

### Display the total profits per category 

In [ ]:
# profits (from products, customers): 5491
SELECT P.product_id, Ct.cart_id, O.order_id, P.price, Cp.quantity, (P.price*Cp.quantity), SUM(P.price*Cp.quantity) OVER () as prof_prod 
FROM
	carts as Ct,
    cart_products as Cp,
	products as P,
	orders as O 
WHERE
	P.product_id = Cp.product_id
    and Cp.cart_id = Ct.cart_id
    and Ct.cart_id = O.cart_id
    
SELECT DISTINCT C.customer_name, Ct.cart_id, O.order_id, Ct.total, SUM(Ct.total) OVER () as prof_cust
	FROM
		customers as C,
		carts as Ct,
		orders as O    
	WHERE
		C.customer_id = Ct.customer_id
		And Ct.cart_id = O.cart_id
	ORDER BY
		C.customer_name

In [ ]:
# amount spend per order, also the price cat
WITH am_per_ord as (
	SELECT C.customer_name, Ct.cart_id, O.order_id, Ct.total
	FROM
		customers as C,
		carts as Ct,
		orders as O    
	WHERE
		C.customer_id = Ct.customer_id
		And Ct.cart_id = O.cart_id
	ORDER BY
		C.customer_name
		),
	price_prod_cat as (
    SELECT 
	P.product_id,
    P.product_name, 
    P.price, 
    CASE WHEN P.price < 33 THEN "Low"
		WHEN P.price > 66 THEN "High"
        ELSE "Medium"
        END AS products_cat
	FROM products AS P
	ORDER BY P.price
    )
SELECT am_per_ord.customer_name, 

# total amount (value) per customer: 5491
SELECT C.customer_name, Ct.cart_id, O.order_id, Ct.total, SUM(Ct.total) OVER (PARTITION BY C.customer_name ) as ord_p_c
	FROM
		customers as C,
		carts as Ct,
		orders as O    
	WHERE
		C.customer_id = Ct.customer_id
		And Ct.cart_id = O.cart_id
        
SELECT DISTINCT C.customer_name, Ct.cart_id, O.order_id, Ct.total, SUM(Ct.total) OVER () as prof
	FROM
		customers as C,
		carts as Ct,
		orders as O    
	WHERE
		C.customer_id = Ct.customer_id
		And Ct.cart_id = O.cart_id
	ORDER BY
		C.customer_name

#total profits per products, per category, total quantity per product send, product profit ranks, total profits per category, total money spend
WITH  sum_cat as (
SELECT P.product_id,Ct.cart_id, O.order_id,  P.price, P.products_cat, Cp.quantity, (P.price*Cp.quantity), SUM(P.price*Cp.quantity) OVER (PARTITION BY product_id) as sum_profit_prod 
FROM
	carts as Ct,
    cart_products as Cp,
	( SELECT
		P.product_id,
		P.product_name, 
		P.price, 
        
		CASE WHEN P.price < 33 THEN "Low"
			WHEN P.price > 66 THEN "High"
			ELSE "Medium"
			END AS products_cat
	FROM products AS P
	ORDER BY P.price) as P,
	orders as O
WHERE
	P.product_id = Cp.product_id
    and Cp.cart_id = Ct.cart_id
    and Ct.cart_id = O.cart_id
 ), qt_prd_ord AS (
	SELECT P.product_id, MAX(SUM(Cp.quantity)) OVER (PARTITION BY P.product_id ORDER BY P.product_id) as ordered_prod
	FROM
		carts as Ct,
		orders as O,
		cart_products as Cp,
		products as P
	WHERE
		Ct.cart_id = O.cart_id
		and P.product_id = Cp.product_id
		and Cp.cart_id = Ct.cart_id
	GROUP BY
		P.product_id
	ORDER BY
		ordered_prod
	)
SELECT 
	sum_cat.product_id, 
    sum_cat.products_cat, 
    sum_cat.sum_profit_prod as tot_prof_prod, 
    qt_prd_ord.ordered_prod,
    RANK () OVER (ORDER BY sum_cat.sum_profit_prod DESC ) as amount_ord_rank,
    SUM(sum_cat.sum_profit_prod) OVER (PARTITION BY sum_cat.products_cat) as cat_tot_pro,
    SUM(sum_cat.sum_profit_prod) OVER () as sum_prof_prod
FROM 
	sum_cat,
    qt_prd_ord
WHERE
	qt_prd_ord.product_id = sum_cat.product_id
GROUP BY 
	sum_cat.product_id, 
    sum_cat.products_cat, 
    tot_prof_prod
ORDER BY 
    tot_prof_prod DESC

#https://stackoverflow.com/questions/41887460/select-list-is-not-in-group-by-clause-and-contains-nonaggregated-column-inc
#OR simply include all columns in the GROUP BY clause that was specified by the SELECT statement. The sql_mode can be left enabled.
